In [1]:
import csv
import time

start = time.time()

with open('wifi_list.csv') as f:
    wifiTable = [{k: str(v) for k, v in row.items()}
        for row in csv.DictReader(f, skipinitialspace=True)]

print "Data Import : " + str(time.time() - start)

Import : 0.0930860042572


In [11]:
## Truncate the table 
conn = pymysql.connect(host='mysql.clarksonmsda.org', \
                       port=3306, \
                       user='ia626', \
                       passwd='ia626clarkson', \
                       db='ia626', autocommit=True)
cur = conn.cursor(pymysql.cursors.DictCursor)

cur.execute("TRUNCATE TABLE `mdhaque_wifi`")
print "Table Truncate : " + str(time.time() - start)

Table Truncate : 177.265734911


In [12]:
## Buffered insert to insert faster
i=0
sql=''
tokens = []
n= 0

sql = '''INSERT INTO mdhaque_wifi (`MAC`,`SSID`,`AuthMode`,`FirstSeen`, 
    `Channel`,`RSSI`,`CurrentLatitude`,`CurrentLongitude`,`AltitudeMeters`,`Type`
    ) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);'''
blocksizes = [1000]
for bs in blocksizes:
    start = time.time()
    for row in wifiTable:
        dtl = row['FirstSeen'].split(' ')
        dts = '0000-00-00 00:00'
        if len(dtl) == 2:
            dl = dtl[0].split("/")
            dts = dl[2] + '/' + dl[0] + '/'+dl[1] + ' ' +dtl[1] 
        tokens.append((row["MAC"],row["SSID"],row["AuthMode"],dts, \
        row["Channel"],row["RSSI"],row["CurrentLatitude"],row["CurrentLongitude"],\
        row["AltitudeMeters"],row["Type"]))
        #print conn.insert_id()
        if i % bs == 0:
            n+=1
            #print tokens
            bstart = time.time()
            cur.executemany(sql,tokens)
            conn.commit()
            #print "Block " + str(n) +" "+ str(time.time() - bstart)
            tokens = []
        i+=1
    print "block size: " + str(bs) + " - total time : " + str(time.time() - start)
    if len(tokens) > 0:
        cur.executemany(sql,tokens)
        conn.commit()
    
cur.close()
conn.close()

block size: 1000 - total time : 1.63105893135


In [13]:
## At the begining if the file does not exist CREATE TABLE
import pymysql # run pip install pymysql if this fails

conn = pymysql.connect(host='mysql.clarksonmsda.org', \
                       port=3306, \
                       user='ia626', \
                       passwd='ia626clarkson', \
                       db='ia626', autocommit=True) #setup credential from config file??
cur = conn.cursor(pymysql.cursors.DictCursor)

cur.execute('''

CREATE TABLE IF NOT EXISTS `mdhaque_wifi` (
  `id` int(5) NOT NULL AUTO_INCREMENT,
  `MAC` varchar(50) NOT NULL,
  `SSID` varchar(50) NOT NULL,
  `AuthMode` varchar(100) NOT NULL,
  `FirstSeen` datetime NOT NULL,
  `Channel` int(3) NOT NULL,
  `RSSI` int(5) NOT NULL,
  `CurrentLatitude` decimal(11,8) NOT NULL,
  `CurrentLongitude` decimal(11,8) NOT NULL,
  `AltitudeMeters` decimal(10,10) NOT NULL,
  `Type` varchar(50) NOT NULL,
  PRIMARY KEY (`id`)
) ENGINE=MyISAM  DEFAULT CHARSET=latin1 AUTO_INCREMENT=4485 ;
''')

cur.close()
conn.close()

/Users/dr.sami/anaconda2/lib/python2.7/site-packages/pymysql/cursors.py:329: Warning: (1050, "Table 'mdhaque_wifi' already exists")
  self._do_get_result()
